In [0]:
# #### JOIN ####
columns = ["RId","RName", "RActive"]
data = [(1, "R1", 1),
    (2, "R2", 1),
    (3, "R3", 1)]
df_role = spark.createDataFrame(data=data, schema=columns)
df_role.createOrReplaceTempView("roles")

columns = ["UId","UName", "UActive"]
data = [(1, "U1", 1),
    (2, "U2", 1),
    (3, "U3", 0),
    (4, "U4", 1)]
df_user = spark.createDataFrame(data=data, schema=columns)
df_user.createOrReplaceTempView("users")

df_role.show()

df_user.show()

columns = ["UId", "RId"]
data = [(1, 1), (1, 2), (2, 3), (3, 1), (3,2), (3,3), (4, 1), (4,3)]
df_user_role = spark.createDataFrame(data=data, schema=columns)
df_user_role.createOrReplaceTempView("user_role")

print("Spark-Sql Approach")
new_df = spark.sql("select u.UName, r.RName, u.UActive, r.RActive from users as u, roles as r, user_role as ur where ur.UId = u.UId and ur.RId = r.RId and UActive=1")
new_df.show()

print("Dataframe-Join Approach")
print("df_user_role.join(df_user)")
df1 = df_user_role.join(df_user, df_user_role.UId == df_user.UId, "inner")
df1.show()

print("user_role_user.join(df_role)")
df2 = df1.join(df_role, df_role.RId == df1.RId, "inner")
df2.show()

In [0]:
#SQL and DF
columns = ["UId","Name", "Zone"]
data = [("1", "User1", "Z1"),
    ("2", "User2", "Z1"),
    ("3", "User3", "Z2")]

df = spark.createDataFrame(data=data, schema=columns)

#Spark DF based
df.show(truncate=False)
df.groupBy("Zone").count().show(truncate=False)

#Spark-Sql Based
df.createOrReplaceTempView("user_zone")
df_sql = spark.sql("SELECT Zone, count(Name) FROM user_zone group by Zone")
df_sql.show()

In [0]:
#UDF - Single Row/Record
#UDF’s a.k.a User Defined Functions
#extend and reuse function in sql

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,sum,avg

def uppercase(str):
    return str.upper() 
  
convertUDF = udf(lambda z: uppercase(z)) 

df.select(convertUDF(col("Name")).alias("Name")).show(truncate=False)

In [0]:
#Aggregations
import pyspark
from pyspark.sql import functions as F
from pyspark.sql import types as T

columns = ["UId","Name", "Zone"]
data = [("1", "User1", "Z1"),
    ("2", "User2", "Z1"),
    ("3", "User3", "Z2")]

df = spark.createDataFrame(data=data, schema=columns)

df.groupBy('Zone').agg(F.collect_list('Name').alias('value_list')).show()


+----+--------------+
Zone| value_list|
+----+--------------+
 Z1|[User1, User2]|
 Z2| [User3]|
+----+--------------+

In [0]:
#UDF Aggregation
import pyspark
from pyspark.sql import functions as F
from pyspark.sql import types as T

columns = ["UId","Name", "Zone"]
data = [("1", "User1", "Z1"),
    ("2", "User2", "Z1"),
    ("3", "User3", "Z2")]

df = spark.createDataFrame(data=data, schema=columns)

def concat(x):
  print("concat:", type(x), x)
  result = ""
  for i in x:
    result = result + "|" + i
  return result

#Function and Return Type
concat_udf = F.udf(concat, T.StringType())
df.groupBy('Zone').agg(concat_udf(F.collect_list('Name')).alias('users_in_zone')).show()


+----+-------------+
Zone|users_in_zone|
+----+-------------+
 Z1| |User1|User2|
 Z2| |User3|
+----+-------------+

In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,sum,avg
from pyspark.sql import functions as F
from pyspark.sql import types as T
import time
import datetime

columns = ["date", "delay", "distance", "origin", "destination"]
data = [(1619124792403, 1, 50, "SFO", "SJC"),
    (1619134792403, 1, 250, "SJO", "LAX"),
    (1619144792403, 1, 60, "LAX", "SDG")]

df = spark.createDataFrame(data=data, schema=columns)
print("df:", df)
df.show()
df.createOrReplaceTempView("distances")

def parseDate(v):
  print("parseDate::type(v):", type(v), ", val:", v)
  dt = datetime.datetime.fromtimestamp(v/1000.0, tz=datetime.timezone.utc)
  #parseDate::type(v): <class 'int'> , val: 1
  return dt
  
parsedate_udf = F.udf(parseDate, T.TimestampType())
spark.udf.register("parsedate_udf", lambda v: parseDate(v) if not v is None else -1 , T.TimestampType())
#1
spark.sql("select delay, parsedate_udf(date), distance, origin, destination  from distances").show()
#2 - since date transformation has predefined function
spark.sql("select delay, date_format('date', 'MM/dd/yyy'), distance, origin, destination  from distances").show()


df: DataFrame[date: bigint, delay: bigint, distance: bigint, origin: string, destination: string]
+-------------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+-------------+-----+--------+------+-----------+
1619124792403| 1| 50| SFO| SJC|
1619134792403| 1| 250| SJO| LAX|
1619144792403| 1| 60| LAX| SDG|
+-------------+-----+--------+------+-----------+

+-----+--------------------+--------+------+-----------+
delay| parsedate_udf(date)|distance|origin|destination|
+-----+--------------------+--------+------+-----------+
 1|2021-04-22 20:53:...| 50| SFO| SJC|
 1|2021-04-22 23:39:...| 250| SJO| LAX|
 1|2021-04-23 02:26:...| 60| LAX| SDG|
+-----+--------------------+--------+------+-----------+

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3119029963728522> in <module> 
 28 spark . sql ( "select delay, parsedate_udf(date), distance, origin, destination from distances" ) . show ( ) 
 29 #2 - since date transformation has predefined function 
 ---> 30 spark . sql ( "select delay, date_format(date, 'MM/dd/yyy'), distance, origin, destination from distances" ) . show ( ) 

 /databricks/spark/python/pyspark/sql/session.py in sql (self, sqlQuery) 
 707 [ Row ( f1 = 1 , f2 = u'row1' ) , Row ( f1 = 2 , f2 = u'row2' ) , Row ( f1 = 3 , f2 = u'row3' ) ] 
 708 """
 --> 709 return DataFrame ( self . _jsparkSession . sql ( sqlQuery ) , self . _wrapped ) 
 710 
 711 @ since ( 2.0 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : cannot resolve 'date_format(distances.`date`, 'MM/dd/yyy')' due to data type mismatch: argument 1 requires timestamp type, however, 'distances.`date`' is of bigint type.; line 1 pos 14;
'Project [delay#592L, unresolvedalias(date_format(date#591L, MM/dd/yyy, Some(Etc/UTC)), None), distance#593L, origin#594, destination#595]
+- SubqueryAlias distances
 +- LogicalRDD [date#591L, delay#592L, distance#593L, origin#594, destination#595], false